In [ ]:
#Loading in Packages and Data

#Importing Packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr
import os; import time
import pickle
import h5py
###############################################################
def coefs(coefficients,degree):
    coef=coefficients
    coefs=""
    for n in range(degree, -1, -1):
        string=f"({coefficients[len(coef)-(n+1)]:.1e})"
        coefs+=string + f"x^{n}"
        if n != 0:
            coefs+=" + "
    return coefs
###############################################################
start_time = time.time();

#Importing Model Data
check=False
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
job_array=False;index_adjust=0

# dx = 1 km; Np = 1M; Nt = 5 min
data=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_1km_5min.nc') #***
parcel=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_pdata_1km_5min_1e6.nc') #***
t_res='5min'; res='1km'
Np_str='1e6'

# # dx = 1km; Np = 50M
# #Importing Model Data
# check=False
# dir2='/home/air673/koa_scratch/'
# data=xr.open_dataset(dir2+'cm1out_1km_1min.nc') #***
# parcel=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_50M.nc') #***
# res='1km'; t_res='1min'; Np_str='50e6'

# # dx = 1km; Np = 100M
# #Importing Model Data
# check=False
# dir2='/home/air673/koa_scratch/'
# data=xr.open_dataset(dir2+'cm1out_1km_1min.nc') #***
# parcel=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_100M.nc') #***
# res='1km'; t_res='1min'; Np_str='100e6'

# # dx = 250 m
# #Importing Model Data
# check=False
# dir2='/home/air673/koa_scratch/'
# data=xr.open_dataset(dir2+'cm1out_250m.nc') #***
# parcel=xr.open_dataset(dir2+'cm1out_pdata_250m.nc') #***
# res='250m'
# Np_str='150e6'

In [ ]:
def check_memory():
    import sys
    ipython_vars = ["In", "Out", "exit", "quit", "get_ipython", "ipython_vars"]
    print("Top 10 objects with highest memory usage")
    # Get a sorted list of the objects and their sizes
    mem = {
        key: round(value/1e6,2)
        for key, value in sorted(
            [
                (x, sys.getsizeof(globals().get(x)))
                for x in globals()
                if not x.startswith("_") and x not in sys.modules and x not in ipython_vars
            ],
            key=lambda x: x[1],
            reverse=True)[:10]
    }
    print({key:f"{value} MB" for key,value in mem.items()})
    print(f"\n{round(sum(mem.values()),2)/1000} GB in use overall")

In [ ]:
############################################################################################

times=data['time'].values/(1e9 * 60); times=times.astype(float);
minutes=1/times[1] #1 / minutes per timestep = timesteps per minute
kms=np.argmax(data['xh'].values-data['xh'][0].values >= 1)

#LOADING CL MAXS FROM CL TRACKING ALGORITHM
folder = '/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/Project_Algorithms/Tracking_Algorithms/'
open_name = folder+f'whereCL_{res}_{t_res}_ALL_CLS.nc'
whereCL=xr.open_dataset(open_name).load()
whereCL=whereCL.isel(time=slice(0,len(data['time'])))
whereCL=whereCL['maxconv_x']
def Get_Conv_X(t,z,y):
    Conv_X_Max=whereCL[t,z,y,:].values
    return Conv_X_Max

In [ ]:
#JOB ARRAY SETUP
job_array=True
if job_array==True:

    num_jobs=60 #how many total jobs are being run? i.e. array=1-100 ==> num_jobs=100 #***
    total_elements=len(parcel['xh']) #total num of variables

    if num_jobs >= total_elements:
        raise ValueError("Number of jobs cannot be greater than or equal to total elements.")
    
    job_range = total_elements // num_jobs  # Base size for each chunk
    remaining = total_elements % num_jobs   # Number of chunks with 1 extra 
    
    # Function to compute the start and end for each job_id
    def get_job_range(job_id):
        job_id-=1
        # Add one extra element to the first 'remaining' chunks
        start_job = job_id * job_range + min(job_id, remaining)
        end_job = start_job + job_range + (1 if job_id < remaining else 0)
    
        if job_id == num_jobs - 1: 
            end_job = total_elements #- 1
        return start_job, end_job
    # def job_testing():
    #     #TESTING
    #     start=[];end=[]
    #     for job_id in range(1,num_jobs+1):
    #         start_job, end_job = get_job_range(job_id)
    #         print(start_job,end_job)
    #         start.append(start_job)
    #         end.append(end_job)
    #     print(np.all(start!=end))
    #     print(len(np.unique(start))==len(start))
    #     print(len(np.unique(end))==len(end))
    # job_testing()
    
    job_id = int(os.environ.get('SLURM_ARRAY_TASK_ID', 0)) #this is the current SBATCH job id
    if job_id==0: job_id=60
    start_job, end_job = get_job_range(job_id)
    index_adjust=start_job
    print(f'start_job = {start_job}, end_job = {end_job}')


In [ ]:
#SUBSETTING PARCEL DATA
parcel=parcel.isel(xh=slice(start_job,end_job))

In [ ]:
# Reading Back Data Later
##############
import h5py
dir2=dir+'Project_Algorithms/Lagrangian_Arrays/'
open_file=dir2+f'lagrangian_binary_array_{res}_{t_res}_{Np_str}.h5'
with h5py.File(open_file, 'r') as f:
    parcel_z = f['z'][:,start_job:end_job]
    parcel_x = f['x'][:,start_job:end_job] #*#
    
    # Load the dataset by its name

    W = f['W'][:,start_job:end_job]
    Z = f['Z'][:,start_job:end_job]
    Y = f['Y'][:,start_job:end_job]
    X = f['X'][:,start_job:end_job]

In [ ]:
# # Reading Back Data Later
# ##############
# import h5py
# dir2=dir+'Project_Algorithms/Lagrangian_Binary_Array/'
# with h5py.File(dir2+f'LFC_LCL_binary_array.h5', 'r') as f:
#     # Load the dataset by its name
#     LFC = f['LFC'][:]
#     LCL = f['LCL'][:]

# Reading Back Data Later
##############
import h5py
dir2=dir+'Project_Algorithms/Lagrangian_Arrays/'
open_file=dir2+f'LFC_LCL_binary_array_{res}_{t_res}_{Np_str}.h5'
with h5py.File(open_file, 'r') as f:
    # Load the dataset by its name
    LFC = f['LFC'][:,start_job:end_job]
    LCL = f['LCL'][:,start_job:end_job]

In [ ]:
check_memory()

In [ ]:
#################################################################

In [ ]:
#Updated Lagrangian Tracking Algorithm

#Algorithm Steps:
#(1) Find the first time a parcel is above the LFC:
#(2) First check if the parcel ascends (w>=0.1) for another 20 minutes
#(3) If so, find first time, the parcel slows down (w<0.1)
#(4) If that time is when the parcel is above 750m, save it, "forget", and move on to next parcel
#(5) If that time is when the parcel is below 750m, check if it is within 2km of the CL_Max found from the CL Tracking Algorithm
#(6) If the parcel is near the CL, store in, otherwise save it, "forget", and move on to next parcel
#(7) Continue to next parcel

#(Also, if during, traceback, the parcel escapes the x or z boundary, "forget" parcel, and move on)

In [ ]:
#Numerical Settings
Nt=len(data['time'])
Np=len(parcel['xh'])
dt=times[1]*60
print(f"(Nt,Np,dt) {Nt,Np,dt}")

#For saving ascend-after-LFC info
ascend_lst=[]
CLmaxheight=750 #750m

#BL slow-down-threshold
w_thresh=0.1

In [ ]:
# #OLD CODE (USE BELOW INSTEAD)
# t_by_t=False#LOAD IN ALL AT ONCE (TOO SLOW)
# # t_by_t=True

# if t_by_t==False:
#     print('loading in lagrangian u and w')
#     parcel_u=parcel['u'].data 
#     parcel_w=parcel['w'].data
#     print('done')

# elif t_by_t==True:
#     Nt = len(parcel['time'])
#     Np = len(parcel['xh'])  
    
#     # Initialize final output array
#     def create_empty():
#         out=np.empty((Nt, Np), dtype=np.float32)
#         return out
#     parcel_u = create_empty()
#     parcel_w = create_empty()
    
#     # Load and process timestep-by-timestep
#     for t in range(Nt):
#         if np.mod(t,50)==0: print(f'time = {t}')
#         parcel_u[t, :] = parcel['u'].isel(time=t).data
#         parcel_w[t, :] = parcel['w'].isel(time=t).data
#     print('done')


In [ ]:
#u,w are now added to Lagrangian_Binary_Array results, no need for code above
# Reading Back Data Later
##############
import h5py
dir2=dir+'Project_Algorithms/Lagrangian_Arrays/'
open_file=dir2+f'lagrangian_binary_array_{res}_{t_res}_{Np_str}.h5'
with h5py.File(open_file, 'r') as f:
    parcel_u = f['u'][:,start_job:end_job]
    parcel_w = f['w'][:,start_job:end_job] #*#

In [ ]:
check_memory() #Needs about 3.25G per job ==> request 8G (actually 12G needed)

In [ ]:
# if ((x + dt*u)==0) or ((z + dt*w)==0)
# u=u[t,Z[t,p],Y[t,p],X[t,p]]; W=W[t,p]
# [u[t,Z[t,p],Y[t,p],X[t,p]] for t in time_arr] >np.max(data['xf'].values) or < np.min(data['xf'].values)
# similarly for w
################################################################################################################
#BOUNDARY-ESCAPE CONDITION
xmin=np.min(data['xf'].values)*1e3
xmax=np.max(data['xf'].values)*1e3
zmin=np.min(data['zf'].values)*1e3
zmax=np.max(data['zf'].values)*1e3

def check_boundary(p,where_BL,above_LFC):
    time_arr=np.arange(where_BL,above_LFC)

    def get_x(t,p):
        # return parcel['x'][t,p].item()
        return parcel_x[t,p] 
    def get_u(t,p):
        # return data['uinterp'].isel(time=t,zh=Z[t,p],yh=Y[t,p],xh=X[t,p]).item() #TESTING
        # return parcel['u'][t,p].item() 
        return parcel_u[t,p]
    def get_z(t,p):
        # return parcel['z'][t,p].item()
        return parcel_z[t,p]
    def get_w(t,p):
        # return data['winterp'].isel(time=t,zh=Z[t,p],yh=Y[t,p],xh=X[t,p]).item()
        # return parcel['w'][t,p].item()
        return parcel_w[t,p]
        

    # x_tend = [get_x(t, p) + dt * get_u(t, z, y, x)   #THIS IS OLD, LESS IDEAL
    #       for (t, z, y, x) in zip(time_arr, Z[time_arr, p], Y[time_arr, p], X[time_arr, p])] 
    # z_tend = [get_z(t, p) + dt * get_w(t, z, y, x)  
    #       for (t, z, y, x) in zip(time_arr, Z[time_arr, p], Y[time_arr, p], X[time_arr, p])] 
    
    x_tend = [get_x(t, p) + dt * get_u(t,p)   
          for (t, z, y, x) in zip(time_arr, Z[time_arr, p], Y[time_arr, p], X[time_arr, p])] 
    z_tend = [get_z(t, p) + dt * get_w(t,p)  
          for (t, z, y, x) in zip(time_arr, Z[time_arr, p], Y[time_arr, p], X[time_arr, p])] 

    x_bound=any(val < xmin or val > xmax for val in x_tend)*1
    z_bound=any(val < zmin or val > zmax for val in z_tend)*1

    out=(x_bound,z_bound)
    if out[0]==1:
        print(f'parcel {p} crossed x-boundary between t={where_BL} and t={above_LFC}')
    elif out[1]==1:
        print(f'parcel {p} crossed z-boundary between t={where_BL} and t={above_LFC}')
    return out
#############################################################################################################

In [ ]:
#############################################################################################################
#The Algorithm

In [ ]:
#Initialize Output Storage Vector

#int 32 can store up to the number 2,147,483,647 
#int 32 has 4 bytes per number, so needs (Np*3)*4 bytes of memory
#Np=125000 ==> (125000*3*4)/(1024**3) = 0.001 GB
#Np=50e6 ==> (50e6*3*4)/(1024**3) = 0.56 GB

out_arr=np.zeros((Np,3),dtype=np.int32) 
save_arr=np.zeros((Np,3),dtype=np.int32) #This one is for saving continued-ascent, slow-below-750m parcels that are not with 2 km of CL
save2_arr=np.zeros((Np,3),dtype=np.int32) #This one is for saving continued-ascent, slow-above-750m parcels

In [ ]:
#1--------------Looping over each parcel
for count,p in enumerate(np.arange(Np)): 

    if np.mod(p,5e4)==0: print(f'current parcel: {p}/{Np}')
    
    W_p = W[:,p]
    LFC_p = LFC[:,p] 
   
    #----FIND WHERE PARCEL IS ABOVE LFC----
    indices = np.where(LFC_p == 1)[0]; above_LFC = indices[0] if indices.size > 0 else -999; #FIRST TIME ABOVE LFC
    if above_LFC ==-999:
        # print(f'parcel {p} never above LFC')
        continue #if the parcel is never above the LFC, skip the parcel
    
    #----CHECK IF ASCENDS FOR >= 20 minutes AFTER LFC----
    ascend_array=W_p[above_LFC+1:]
    indices=np.where(ascend_array==0)[0]; ascend_stop=indices[0] if indices.size > 0 else 10000; #location of where parcel stops ascending (labeled 10000 to mark for future analysis)
    ascend_lst.append(ascend_stop) #(also store for histogram)
    if ascend_stop>=20*minutes:
    
        #----FIND THE FIRST TIME W_p<=w_thresh----
        indices=np.where(W_p[0:above_LFC]<w_thresh)[0]
        where_BL=indices[-1] if indices.size > 0 else -999 #FIRST PRIOR TIME W<0.1 (IN THE BL) (ADDED 1 TO GET TIME RIGHT AFTER INTERACTION)
        if where_BL ==-999:
            # print(f'parcel {p} w is never below threshold prior to t={above_LFC}')
            continue #if the parcel never slows down backwards in time (unlikely), skip the parcel
            
        #check for boundary escapes
        ################################
        future_location=check_boundary(p,where_BL,above_LFC)
        if (future_location[0]+future_location[1]>=1): continue #if parcel crosses boundary, skips current parcel
        ################################
        
        #----CHECK IF PARCEL SLOWED DOWN LOW ENOUGH----
        if parcel_z[where_BL,p]<=CLmaxheight: #PARCEL MUST BE BELOW 750m WHEN CONTACTING CL #***
        # if LCL[where_BL,p]==0: #PARCEL MUST BE BELOW LCL WHEN CONTACTING CL (not recommended)
    
            #----CHECK IF CL IS WITHIN 2km----
            #Find the CL-max x-location
            t=where_BL; z=Z[where_BL,p]; y=Y[where_BL,p]; x=X[where_BL,p]
            CONV_X=Get_Conv_X(t,z,y)
            within_CL=np.any(np.isin(CONV_X, np.arange(x-2*kms,x+3*kms)))
            
            if within_CL==True:
                #save X's (t,p) 
                print(f'Parcel {p} is success at time {where_BL}')
                out_arr[p,0]=p
                out_arr[p,1]=where_BL
                out_arr[p,2]=above_LFC 
            else: #continued-ascent, slow-below-750m parcels that are not with 2 km of CL
                #SAVE PARCEL
                # print(f'Parcel {p} not near CL at t={where_BL}')
                save_arr[p,0]=p
                save_arr[p,1]=where_BL
                save_arr[p,2]=above_LFC 
    
        else: #continued-ascent, slow-above-750m parcels
            #SAVE PARCEL
            # print(f'Parcel {p} above {CLmaxheight}m at t={where_BL}')
            save2_arr[p,0]=p
            save2_arr[p,1]=where_BL
            save2_arr[p,2]=above_LFC         
            
        #END OF LOOP, THEN WE MOVE ON TO NEXT PARCEL p

In [ ]:
#CORRECTING DATA PARCEL ID BASED ON JOB NUMBER
#####################################################
out_arr[np.where(np.any(out_arr != 0, axis=1))[0],0]+=index_adjust #*needed for job array*+=index_adjust #*needed for job array*
save_arr[np.where(np.any(save_arr != 0, axis=1))[0],0]+=index_adjust #*needed for job array*+=index_adjust #*needed for job array*
save2_arr[np.where(np.any(save2_arr != 0, axis=1))[0],0]+=index_adjust #*needed for job array*+=index_adjust #*needed for job array*

In [ ]:
#REMOVING BLANK ROWS
#####################################################
def remove_zero_rows(arr):
    arr = arr[~np.all(arr == 0, axis=1)]
    return arr
out_arr=remove_zero_rows(out_arr)
save_arr=remove_zero_rows(save_arr)
save2_arr=remove_zero_rows(save2_arr)

In [ ]:
# #Storing output and save data
#USES H5 FILE
###################################################################################################################################
out_file = dir + f'Project_Algorithms/Tracking_Algorithms/trackout/parcel_tracking_{res}_{t_res}_{Np_str}_{job_id}.h5'
with h5py.File(out_file, 'w') as hf:
    hf.create_dataset('out_arr', data=out_arr, compression="gzip")
    hf.create_dataset('save_arr', data=save_arr, compression="gzip")
    hf.create_dataset('save2_arr', data=save2_arr, compression="gzip")

end_time = time.time(); elapsed_time = end_time - start_time; print(f"Total Elapsed Time: {elapsed_time} seconds")  

In [ ]:
check_memory()

In [ ]:
# #Storing output and save data
# #OLD, USES NETCDF, H5 IS RECOMMENDED INSTEAD HERE
# ###################################################################################################################################
# ds=xr.Dataset({
#     'out_arr': (['rows', 'columns'], out_arr.astype(float)),
#     'save_arr': (['rows', 'columns'], save_arr.astype(float)),
#     'save2_arr': (['rows', 'columns'], save2_arr.astype(float)),
# })


# out_file = dir+f'Project_Algorithms/Tracking_Algorithms/trackout/parcel_tracking_{res}_{t_res}_{Np_str}_{job_id}.nc'
# ds.to_netcdf(out_file)
# ###########################################################################################################################
# end_time = time.time(); elapsed_time = end_time - start_time; print(f"Total Elapsed Time: {elapsed_time} seconds")  

In [ ]:
############################################################
#Run after finishing job_array
recombine=False #KEEP FALSE WHEN JOB_ARRAY IS RUNNING
# recombine=True 

In [ ]:
# #OLD USING NC
# if recombine==True:
#     #combine all job output arrays 
#     import numpy as np
#     import matplotlib.pyplot as plt
#     import xarray as xr
#     import os; import time
#     dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
    
#     num_jobs=60#350 #***
#     for job_id in range(1, num_jobs+1):
#         if np.mod(job_id,50)==0: print(f'job_id = {job_id}')
#         # Open the dataset and append it to the list
#         if job_id == 1: 
#             in_file=dir+f'Project_Algorithms/Tracking_Algorithms/trackout/parcel_tracking_{res}_{t_res}_{Np_str}_{job_id}.h5'
            
#             out_arr = xr.open_dataset(in_file)['out_arr']
#             save_arr = xr.open_dataset(in_file)['save_arr']
#         elif job_id >= 2: 
#             in_file = dir+f'Project_Algorithms/Tracking_Algorithms/trackout/parcel_tracking_{res}_{t_res}_{Np_str}_{job_id}.h5'
            
#             out2 = xr.open_dataset(in_file)['out_arr']
#             save2 = xr.open_dataset(in_file)['save_arr']
#             out_arr=np.concatenate((out_arr, out2), axis=0)
#             save_arr=np.concatenate((save_arr, save2), axis=0)
#     ds=xr.Dataset({
#         'out_arr': (['rows', 'columns'], out_arr.astype(float)),
#         'save_arr': (['rows', 'columns'], save_arr.astype(float)),
#     })
#     out_file=dir+f'Project_Algorithms/Tracking_Algorithms/parcel_tracking_{res}_{t_res}_{Np_str}.h5'
#     ds.to_netcdf(out_file)

In [ ]:
#combine all job output arrays 
if recombine == True:
    num_jobs = 350 #60
    var_names = ['out_arr', 'save_arr', 'save2_arr']
    recombined_arrays = {}  # Store final arrays here

    def MakeUnique(arr):
        return np.unique(arr, axis=0)
    def get_total_count(var_name):
        return sum(
            h5py.File(dir + f'Project_Algorithms/Tracking_Algorithms/trackout/parcel_tracking_{res}_{t_res}_{Np_str}_{job_id}.h5', 'r')[var_name].shape[0]
            for job_id in range(1, num_jobs + 1)
        )

    def load_file(job_id, var_name):
        in_file = dir + f'Project_Algorithms/Tracking_Algorithms/trackout/parcel_tracking_{res}_{t_res}_{Np_str}_{job_id}.h5'
        with h5py.File(in_file, 'r') as hf:
            arr = hf[var_name][:]
        return arr

    # Preallocate arrays
    for var_name in var_names:
        total_count = get_total_count(var_name)
        recombined_arrays[var_name] = np.zeros((total_count, 3), dtype=np.int32)

    # Fill arrays
    for var_name in var_names:
        print(f"Combining data for {var_name}")
        left_ind = 0
        for job_id in range(1, num_jobs + 1):
            if job_id % 50 == 0:
                print(f"{var_name}: processing job {job_id}")
            arr = load_file(job_id, var_name)
            n_rows = arr.shape[0]; right_ind = left_ind + n_rows
            recombined_arrays[var_name][left_ind:right_ind, :] = arr
            left_ind = right_ind

    #Make Unique
    for var_name in var_names:
        recombined_arrays[var_name]=MakeUnique(recombined_arrays[var_name])
        
    # Write to file
    out_file = dir + f'Project_Algorithms/Tracking_Algorithms/parcel_tracking_{res}_{t_res}_{Np_str}.h5'
    with h5py.File(out_file, 'w') as hf:
        for var_name in var_names:
            
            hf.create_dataset(var_name, data=recombined_arrays[var_name], compression="gzip")

In [ ]:
# #LOADING BACK IN
# def load_file():
#     in_file=dir+f'Project_Algorithms/Tracksing_Algorithms/parcel_tracking_{res}_{t_res}_{Np_str}.h5'
#     with h5py.File(in_file, 'r') as hf:
#         out_arr=hf['out_arr'][:]
#         save_arr=hf['save_arr'][:]
#         save2_arr=hf['save2_arr'][:]
#     return out_arr,save_arr,save2_arr
# [out_arr,save_arr,save2_arr]=load_file()

In [ ]:
# #TESTING
# out_file = dir + f'Project_Algorithms/Tracking_Algorithms/parcel_tracking_{res}_{t_res}_{Np_str}.h5'
# with h5py.File(out_file, 'r') as hf:
#     test=hf['out_arr'][0:232]
# np.all(test==out_arr)